**`Análisis exploratorio de los datos`**: _(Exploratory Data Analysis-EDA)_

Primero importamos las librerias que  vamos a necesitar

In [2]:
import pandas as pd
import matplotlib.pyplot as plt

Cargamos los archivos .csv a los que ya se les aplico las transformaciones

In [3]:
df_amazon_prime = pd.read_csv('../dataset_ETL/amazon_prime_ETL.csv', delimiter = ',',encoding = "utf-8")
df_disney_plus = pd.read_csv('../dataset_ETL/disney_plus_ETL.csv',delimiter= ',', encoding = "utf-8" )
df_hulu = pd.read_csv('../dataset_ETL/hulu_ETL.csv',delimiter= ',', encoding = "utf-8" )
df_netflix = pd.read_csv('../dataset_ETL/netflix_ETL.csv',delimiter= ',', encoding = "utf-8" )
df_ratings = pd.read_csv('../dataset_ETL/ratings_ETL.csv', delimiter=',', encoding= "utf-8")

Se corre un sample en cada dataframe para ver que tipo de datos hay

In [ ]:
df_amazon_prime.sample(20)

In [ ]:
df_disney_plus.sample(20)

In [ ]:
df_hulu.sample(20)

In [ ]:
df_netflix.sample(20)

In [ ]:
df_ratings.sample(20)

Observamos la informacion detallada de cada dataframe

In [ ]:
df_amazon_prime.info()

In [ ]:
df_disney_plus.info()

In [ ]:
df_hulu.info()

In [ ]:
df_netflix.info()

Se analizaran los valores faltantes

In [ ]:
df_amazon_prime.isnull().sum()

In [ ]:
df_disney_plus.isnull().sum()

In [ ]:
df_hulu.isnull().sum()

In [ ]:
df_netflix.isnull().sum()

Se analizaran las columnas con valores numericos

Como a la columna 'date_added' no lo toma con formato DATETIME, se convierte.

In [14]:
df_amazon_prime['date_added'] = pd.to_datetime(df_amazon_prime['date_added'])
df_disney_plus['date_added'] = pd.to_datetime(df_disney_plus['date_added'])
df_hulu['date_added'] = pd.to_datetime(df_hulu['date_added'])
df_netflix['date_added'] = pd.to_datetime(df_netflix['date_added'])

In [12]:
df_amazon_prime['date_added'].min()

Timestamp('2021-03-30 00:00:00')

In [15]:
df_disney_plus['date_added'].min()

Timestamp('2019-10-01 00:00:00')

In [16]:
df_hulu['date_added'].min()

Timestamp('2006-07-14 00:00:00')

In [42]:
df_ratings['timestamp'].min()

'1995-01-09'

Se analizaran si hay duplicados

In [61]:
df_ratings.duplicated().value_counts()

False    11013823
True        10466
dtype: int64

Se utilizara un diagrama de caja para identificar valores atípicos en la columna "scored" del dataframe ratings

In [60]:
q1 = df_ratings['scored'].quantile(0.25)
q3 = df_ratings['scored'].quantile(0.75)
iqr = q3 - q1

# define los límites para los outliers
lower_limit = q1 - 1.5 * iqr
upper_limit = q3 + 1.5 * iqr

# identifica los outliers
outliers = df_ratings[(df_ratings['scored'] < lower_limit) | (df_ratings['scored'] > upper_limit)]

# muestra los outliers
print(outliers)

          userid  scored   timestamp movieid  scoreds
0              1     1.0  2015-03-09   as680      3.5
24             1     0.5  2015-03-09  as7193      3.5
42             2     1.0  1997-06-23  hs1914      3.6
43             2     1.0  1997-06-23  as9455      3.5
84             4     1.0  2003-01-15  ns3279      3.6
...          ...     ...         ...     ...      ...
11024167  124373     1.0  2001-05-31  ns7039      3.5
11024171  124374     1.0  2001-11-28  ds1361      3.6
11024179  124374     1.0  2001-11-28  as9146      3.6
11024192  124375     1.0  1996-08-14   hs282      3.4
11024218  124376     1.0  2001-04-26  ns5214      3.6

[514764 rows x 5 columns]


El resultado que te devolvió es un subconjunto de datos de la tabla que compartiste anteriormente. En este subconjunto, se han eliminado todas las filas donde los valores de la columna "scoreds" se encuentran fuera del rango intercuartil, lo que sugiere que son posibles valores atípicos o "outliers". En otras palabras, estos son los datos que no son atípicos en la columna "scoreds". La tabla resultante tiene 514764 filas y 5 columnas. Cada fila contiene información de un usuario que ha calificado una película, incluyendo el ID del usuario, su calificación ("scored"), la fecha en que se hizo la calificación ("timestamp"), el ID de la película ("movieid") y el promedio de todas las calificaciones para esa película ("scoreds").